In [4]:
%pip install psycopg2-binary
%pip install pandas
%pip install numpy
%pip install scipy
%pip install ipywidgets
%pip install requests
%pip install boto3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Metadata Wide Association Study (MWAS)
- statistical method to identify associations between BioSample metadata and provided count data
- inspired by GWAS (Genome-Wide Association Study) 

In [3]:
# import required libraries
import psycopg2
import pandas as pd
import numpy as np
import scipy.stats as stats
import requests
import warnings
import logging
import sys
import io
import pickle
import csv
import boto3

In [5]:
# --------------- HELPER FUNCTIONS ---------------
# - specific to the rfamily run

def family_biosamples_mapper(row):
    if row['bio_sample'] in family_biosamples_n_reads:
        family_biosamples_n_reads[row['bio_sample']].append(row['n_reads'])
        family_biosample_spots[row['bio_sample']].append(row['spots'])
        # family_biosample_runs[row['bio_sample']].append(row['run_id']) # TO BE USED IF MAPPING IS DONE FROM RUNS
    else:
        family_biosamples_n_reads[row['bio_sample']] = [row['n_reads']]
        family_biosample_spots[row['bio_sample']] = [row['spots']]
        # family_biosample_runs[row['bio_sample']] = [row['run_id']] # TO BE USED IF MAPPING IS DONE FROM RUNS

    
def family_bioproject_mapper(row):
    try:
        return srarun_bioprojects_map[row['bio_sample']]
    except:
        return np.nan

def get_n_reads(row):
    # try to get the n_reads from family_biosamples_n_reads, then from srarun_biosamples_map, then default to np.nan
    #   - tries to get exact n_reads from serratus
    #   - if run in serratus but no n_reads, then set to 0 (from srarun_biosamples_map)
    #   - if not run in serratus, then set to np.nan
    try:
        return family_biosamples_n_reads[row['biosample_id']]
    except:
        try:
            return srarun_biosamples_map[row['biosample_id']]
        except:
            return np.nan

def get_n_spots(row):
    # try to get the n_spots from family_biosample_spots, otherwise default to 0 
    ##### CHECK THIS ##### will have divide by 0 error if the mapping is wrong
    try:
        return family_biosample_spots[row['biosample_id']]
    except:
        return 0

def get_log_fold_change(true, false):
    # calculate the log fold change of true with respect to false
    #   - if true and false is 0, then return 0

    if true == 0 and false == 0:
        return 0
    elif true == 0:
        return -np.inf
    elif false == 0:
        return np.inf
    else:
        return np.log2(true / false)

# ---------------------------------------------

In [7]:
# load information from pickles
# ADJUST FILE PATH ON FINAL RUN
with open(f'pickles/rfam_df.pickle', 'rb') as f:
    rfam_df = pickle.load(f)
with open(f'pickles/srarun_df.pickle', 'rb') as f:
    srarun_df = pickle.load(f)

with open(f'pickles/srarun_biosamples_map.pickle', 'rb') as handle:
    # mapping from biosamples (in srarun_df) to 0 for default count
    srarun_biosamples_map = pickle.load(handle) 
with open(f'pickles/srarun_bioprojects_map.pickle', 'rb') as handle:
    # mapping from biosamples (in srarun_df) to bioprojects
    srarun_bioprojects_map = pickle.load(handle)

In [2]:
import csv

with open('names.csv', 'w', newline='') as csvfile:
    fieldnames = ['first_name', 'last_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})
    writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
    writer.writerow({'last_name': 'Spam','first_name': 'Wonderful'})